## Canberra lithologies case study

Motivated by learning that the ACT is interested in managed aquifer recharge for watering some green spaces.

This notebook does not look at AEM data although sitting under a repository suggesting so. 

## Downloading the data 

Not throughly documented.

Data was downloaded from the usual places, NGIS and Elvis. NGIS when using the Murrumbidgee catchment was actually not including the bores in the ACT, so needed to download the ACT ones also, and this present notebook will do the merging of the lithology logs. Spatial locations were merged manually, and subsetted, in QGIS

Some of the data output by this present notebook fed into a [lithology log viewer](https://github.com/csiro-hydrogeology/lithology-viewer) that can be run as a dashboard on Binder.


In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import geopandas as gpd


In [ ]:
# Only set to True for co-dev of ela from this use case:
ela_from_source = False
ela_from_source = True

In [ ]:
if ela_from_source:
    if ('ELA_SRC' in os.environ):
        root_src_dir = os.environ['ELA_SRC']
    elif sys.platform == 'win32':
        root_src_dir = r'C:\src\github_jm\pyela'
    else:
        username = os.environ['USER']
        root_src_dir = os.path.join('/home', username, 'src/ela/pyela')
    pkg_src_dir = root_src_dir
    sys.path.insert(0, pkg_src_dir)

from ela.textproc import *
from ela.utils import *
from ela.classification import *
from ela.visual import *
from ela.spatial import SliceOperation

## Importing data

There are two main sets of information we need: the borehole lithology logs, and the spatial information in the surface elevation (DEM) and geolocation of a subset of bores around Bungendore. 

In [ ]:
data_path = None

You probably want to explicitly set `data_path` to the location where you put the folder(s) e.g:

In [ ]:
#data_path = '/home/myusername/data' # On Linux, if you now have the folder /home/myusername/data/Bungendore
#data_path = r'C:\data\Lithology'  # windows, if you have C:\data\Lithology\Bungendore

Otherwise a fallback for the pyela developer(s)

In [ ]:
if data_path is None:
    if ('ELA_DATA' in os.environ):
        data_path = os.environ['ELA_DATA']
    elif sys.platform == 'win32':
        data_path = r'C:\data\Lithology'
    else:
        username = os.environ['USER']
        data_path = os.path.join('/home', username, 'data')

In [ ]:
data_path

In [ ]:
cbr_datadir = os.path.join(data_path, 'Brisbane')
cbr_datadir_out = os.path.join(cbr_datadir, 'out')
ngis_datadir = os.path.join(data_path, 'NGIS')
act_shp_datadir = os.path.join(ngis_datadir, 'shp_ACT')
bidgee_shp_datadir = os.path.join(ngis_datadir, 'shp_brisbane_river')

In [ ]:
import pickle

interp_litho_filename = os.path.join(cbr_datadir_out,'3d_primary_litho.pkl')
with open(interp_litho_filename, 'rb') as handle:
    lithology_3d_array = pickle.load(handle)

    
fp = os.path.join(cbr_datadir_out, 'dem_array_data.pkl')
with open(fp, 'rb') as handle:
    dem_array_data = pickle.load(handle)

    
classified_logs_filename = os.path.join(cbr_datadir_out, 'classified_logs.pkl')
with open(classified_logs_filename, 'rb') as handle:
    df = pickle.load(handle)



In [ ]:
# more classes for display of raw logs
lithologies = ['shale', 'clay','granite','soil','sand', 'porphyry','siltstone', 'dacite', 'rhyodacite', 'gravel', 'limestone', 'sandstone', 'slate', 'mudstone', 'rock', 'ignimbrite', 'tuff']
# Prep for visualisation
lithology_color_names = [
    'lightslategrey', # Shale
    'olive', # clay
    'dimgray', # granite
    'chocolate',  # soil
    'gold', # sand
    'tomato', # porphyry
    'teal', # siltstone
    'darkgrey', # dacite
    'whitesmoke', # rhyodacite
    'powderblue', # gravel 
    'yellow', #limestone
    'papayawhip', #sandstone
    'dimgray', #slate
    'darkred', #mudstone
    'grey', #rock
    'khaki', #ignimbrite
    'lemonchiffon' #tuff
]

In [ ]:
# TODO: how could this information be serialised/reloaded between notebooks?
# (442.0, 824.0)
ahd_min=442
ahd_max=824

z_ahd_coords = np.arange(ahd_min,ahd_max,1)
dim_x,dim_y,dim_z = lithology_3d_array.shape
dims = (dim_x,dim_y,dim_z)

In [ ]:
z_index_for_ahd = z_index_for_ahd_functor(b=-ahd_min)

## 2D visualisations

In [ ]:
lithology_cmap = discrete_classes_colormap(lithology_color_names) # Later for exporting to RGB geotiffs??
litho_legend_display_info = [(lithology_cmap[i], lithologies[i], lithology_color_names[i]) for i in range(len(lithologies))]

In [ ]:
litho_legend = legend_fig(litho_legend_display_info)

In [ ]:
cms = cartopy_color_settings(lithology_color_names)

In [ ]:
z = z_index_for_ahd(ahd_min + 20)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

imgplot = plt.imshow(to_carto(lithology_3d_array[:, :, z]), cmap=cms['cmap'])
title = plt.title('Primary litho at +YYYmAHD')

## 3D visualisation

In [ ]:
from ela.visual3d import *

In [ ]:
xx, yy = dem_array_data['mesh_xy']

In [ ]:
from mayavi import mlab

In [ ]:
vis_litho = LithologiesClassesVisual3d(lithologies, lithology_color_names, 'black')

In [ ]:
# TODO: problematic with this data - investigate
# vis_litho.render_classes_planar(lithology_3d_array, 'Primary lithology')

In [ ]:
# vis_litho.render_class(lithology_3d_array, 0)

ela has facilities to visualise overlaid information: DEM, classified bore logs, and volumes of interpolated lithologies. This is important to convey .

First a bit of data filling for visual purposes, as NaN lithology class codes may cause issues.

In [ ]:
df_infilled = df.fillna({PRIMARY_LITHO_NUM_COL: -1.0})
df_infilled = df_infilled[(df_infilled[DEPTH_TO_AHD_COL] > (ahd_min-20))]

In [ ]:
# A factor to apply to Z coordinates, otherwise things would be squashed visually along the heights.
# Would prefer a visual only scaling factor, but could not find a way to do so. 
Z_SCALING = 20.0

In [ ]:
z_coords = np.arange(ahd_min,ahd_max,1)

In [ ]:
overlay_vis_litho = LithologiesClassesOverlayVisual3d(lithologies, lithology_color_names, 'black', dem_array_data, z_coords, Z_SCALING, df_infilled, PRIMARY_LITHO_NUM_COL)

In [ ]:
def view_class(value):
    f = overlay_vis_litho.view_overlay(value, lithology_3d_array)
    return f

In [ ]:
f = view_class(0.0)

In [ ]:
f = view_class(7.0)

In [ ]:
f = view_class(1.0)

In [ ]:
f = view_class(5.0)

In [ ]:
vis_litho = LithologiesClassesVisual3d(lithologies, lithology_color_names, 'black')

In [ ]:
vis_litho.render_classes_planar(lithology_3d_array, 'Primary lithology')

### Export volumes for depth from ground

In [ ]:
max_depth = 100
dim_x,dim_y = (lithology_3d_array.shape[0],lithology_3d_array.shape[1])
dim_z = max_depth + 1
dims = (dim_x,dim_y,dim_z)

In [ ]:
grid_res = dem_array_data['grid_res']
x_min, x_max, y_min, y_max = dem_array_data['bounds']
xx, yy = dem_array_data['mesh_xy']
dem_array = dem_array_data['dem_array']

In [ ]:
litho_classes_depth=np.empty(dims)
for depth in range(0, dim_z, 1):
    s = slice_volume(lithology_3d_array, dem_array - depth, z_index_for_ahd)
    litho_classes_depth[:,:,(max_depth - depth)] = s

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
imgplot = plt.imshow(to_carto(litho_classes_depth[:,:,dim_z-30]), cmap=cms['cmap'])
t = plt.title('Primary lithologies at 30m depth')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
imgplot = plt.imshow(to_carto(litho_classes_depth[:,:,dim_z-1]), cmap=cms['cmap'])
t = plt.title('Primary lithologies at 1m depth')

## Appendix / Attic

In [ ]:
def plot_borehole_data(df):
    # We may have something fancy visual down the track, for now, a dataframe subsetting. 
    px = [p.Time for p in points]
    py = [p.CND_011 for p in points]

    x_scale, y_scale = LinearScale(), LogScale()
    x_scale.allow_padding = False
    x_ax = Axis(label='Time (s)', scale=x_scale)
    y_ax = Axis(label='CND 011(?)', scale=y_scale, orientation='vertical')

    lines = Lines(x=px, y=py, scales={'x': x_scale, 'y': y_scale})

    elevation = Figure(title='CND 011 Chart', axes=[x_ax, y_ax], marks=[lines])
    elevation.layout.width = 'auto'
    elevation.layout.height = 'auto'
    elevation.layout.min_height = '500px'

    elevation.interaction = IndexSelector(scale=x_scale)

    return elevation

In [ ]:
def link_geo_borehole(geomap, boreholelayer):
    """
    Links the geolocation of the markers to the display of the bvorehole log
    Changing the selection on the marker will update the
    borehole display
    """
    # add a checkbox to auto center
    autocenter = Checkbox(value=False, description='Auto Center')
    autocenter_control = WidgetControl(widget=autocenter, position='bottomright')
    geomap.add_control(autocenter_control)

    brushintsel = geomap.interaction
    def update_range(change):
        """
        Update the position on the map when the elevation
        graph selector changes
        """
        if brushintsel.selected.shape != (1,):
            return
        marker.visible = True
        selected = brushintsel.selected # time stamp in seconds for a day
        point = find_point(selected)
        marker.location = (point.Latitude, point.Longitude)
        if autocenter.value:
            trace.center = marker.location
        #position = max(0, int((selected / distance_from_start) * len(points)))
    brushintsel.observe(update_range, 'selected')

    
def link_trace_elevation(trace, elevation, points):
    """
    Link the trace the elevation graph.
    Changing the selection on the elevation will update the
    marker on the map
    """
    times = np.asarray([p.Time for p in points])

    def find_point(time):
        """
        Find a point given the time
        """
        dist_1 = abs(times - time)
        pos = np.argmin(dist_1)
        return points[pos]
    
    # add a checkbox to auto center
    autocenter = Checkbox(value=False, description='Auto Center')
    autocenter_control = WidgetControl(widget=autocenter, position='bottomright')
    trace.add_control(autocenter_control)
    # mark the current position on the map
    start = points[0]
    marker = CircleMarker(visible=False, location=(start.Latitude, start.Longitude),
                          radius=10, color="green", fill_color="green")
    trace.add_layer(marker)
    brushintsel = elevation.interaction
    def update_range(change):
        """
        Update the position on the map when the elevation
        graph selector changes
        """
        if brushintsel.selected.shape != (1,):
            return
        marker.visible = True
        selected = brushintsel.selected # time stamp in seconds for a day
        point = find_point(selected)
        marker.location = (point.Latitude, point.Longitude)
        if autocenter.value:
            trace.center = marker.location
        #position = max(0, int((selected / distance_from_start) * len(points)))
    brushintsel.observe(update_range, 'selected')


In [ ]:
from ipywidgets import FloatSlider
interact(slow_function,i=FloatSlider(min=1e5, max=1e7, step=1e5));

In [ ]:
def plot_gpx(points):
    trace = plot_map(points)
    elevation = plot_elevation(points)
    debug = Label(value='')
    display(trace)
    display(elevation)
    display(debug)
    link_trace_elevation(trace, elevation, points)

In [ ]:
plot_gpx(points)

## Trying to revisit cartopy... 

In [ ]:

# These functions were an attempt to have interactive maps with ipywidgets but proved to be a pain. 
# I may revisit later on but these are parked. 

def plot_lithologydata_slice_points_redo(df, 
    slice_depth, extent, data_proj, 
    near_field_extents, geoms, terrain):
    # fig,ax=plt.subplots(1,1,figsize=(15,15), subplot_kw={'projection': data_proj, 'extent': extent})
    fig,ax=plt.subplots(1,1,figsize=(15,15), subplot_kw={'projection': data_proj})
    # fig.clear()
    # ax.clear()
    ax.add_image(terrain, 11)
    #ax.add_geometries(geoms[0], ccrs.PlateCarree(),facecolor='none',edgecolor='k',zorder=1)
    #ax.add_geometries(geoms[1], ccrs.PlateCarree(),facecolor='none',edgecolor='r',zorder=1)
    for val,label in zip(ax.get_xticks(), ax.get_xticklabels()):
        label.set_text(str(val))
        label.set_position((val,0))  
    for val,label in zip(ax.get_yticks(), ax.get_yticklabels()):   
        label.set_text(str(val))
        label.set_position((0,val))  
    plt.tick_params(bottom=True,top=True,left=True,right=True,labelbottom=True,labeltop=False,labelleft=True,labelright=False)
    ax.xaxis.set_visible(True)
    ax.yaxis.set_visible(True)
    ax.ticklabel_format(useOffset=False)
    ax.ticklabel_format(style='plain')
    ax.grid(False)
    ax.text(0.1, 0.9, u'\u25B2 \nN ',
        horizontalalignment='center',
        verticalalignment='center',
        fontsize=25, 
        color='k',
        family='Arial',
        transform=ax.transAxes)
    ax.set_extent(near_field_extents, crs=data_proj)
    # Note that all of the above is independent of slice depth and background that would not need redoing
    # but Matplotlib befuddles (or rather the interplay with ipywidgets)
    df_slice=df.loc[(df[DEPTH_FROM_COL] <= slice_depth) & (df[DEPTH_TO_COL] >= slice_depth)]
    ax.scatter(df_slice.Easting.values,df_slice.Northing.values)
    plt.title('bore log locations at %s m depth'%(slice_depth), fontsize=20, weight='bold')
    # I cannot fathom why this stuff actually plots anything 
    # via ipywidgets or otherwise since it returns nothing.

def create_background(extent, data_proj, 
    near_field_extents, geoms):
    fig,ax=plt.subplots(1,1,figsize=(15,15), subplot_kw={'projection': data_proj,'extent': extent})
    stamen_terrain = cimgt.Stamen('terrain-background')
    ax.add_image(stamen_terrain, 11)
    ax.add_geometries(geoms[0], ccrs.PlateCarree(),facecolor='none',edgecolor='k',zorder=1)
    ax.add_geometries(geoms[1], ccrs.PlateCarree(),facecolor='none',edgecolor='r',zorder=1)
    for val,label in zip(ax.get_xticks(), ax.get_xticklabels()):
        label.set_text(str(val))
        label.set_position((val,0))  
    for val,label in zip(ax.get_yticks(), ax.get_yticklabels()):   
        label.set_text(str(val))
        label.set_position((0,val))  
    plt.tick_params(bottom=True,top=True,left=True,right=True,labelbottom=True,labeltop=False,labelleft=True,labelright=False)

    ax.xaxis.set_visible(True)
    ax.yaxis.set_visible(True)
    ax.ticklabel_format(useOffset=False)
    ax.ticklabel_format(style='plain')

    ax.grid(False)

    ax.text(0.1, 0.9, u'\u25B2 \nN ',
        horizontalalignment='center',
        verticalalignment='center',
        fontsize=25, 
        color='k',
        family='Arial',
        transform=ax.transAxes)

    ax.set_extent(near_field_extents, crs=data_proj)
    scatter_layer = ax.scatter(near_field_extents[0], near_field_extents[2])
    return (fig, scatter_layer)

def plot_lithologydata_slice_points(df, slice_depth, scatter_layer, fig):
    df_slice=df.loc[(df[DEPTH_FROM_COL] <= slice_depth) & (df[DEPTH_TO_COL] >= slice_depth)]
    plt.title('bore log locations at %s m depth'%(slice_depth), fontsize=20, weight='bold')
    e = df_slice.Easting.values
    n = df_slice.Northing.values
    bore_coords = [[e[i], n[i]] for i in range(0, len(e))]
    scatter_layer.set_offsets(bore_coords)
    fig.canvas.draw()
    fig.canvas.flush_events()
    return fig

def plot_lithologydata_slice_depth(df, slice_depth, n_neighbours, extent, data_proj, near_field_extents, geoms, gw_subareas, cmap_settings):
    df_slice=df.loc[(df[DEPTH_FROM_AHD_COL] >= slice_depth) & (df[DEPTH_TO_AHD_COL] <= slice_depth)]
    _,ax=plt.subplots(1,1,figsize=(15,15),subplot_kw={'projection': data_proj,'extent': extent})
    stamen_terrain = cimgt.Stamen('terrain-background')
    ax.add_image(stamen_terrain, 11)
    ax.add_geometries(geoms[0], ccrs.PlateCarree(),facecolor='none',edgecolor='k',zorder=1)
    ax.add_geometries(geoms[1], ccrs.PlateCarree(),facecolor='none',edgecolor='r',zorder=1)

    for i, txt in enumerate(df_slice[PRIMARY_LITHO_COL].values):
        plt.annotate(txt,(df_slice.Easting.values[i],df_slice.Northing.values[i]),fontsize=8,clip_on=True)
    
    for val,label in zip(ax.get_xticks(), ax.get_xticklabels()):
        label.set_text(str(val))
        label.set_position((val,0))  
    
    for val,label in zip(ax.get_yticks(), ax.get_yticklabels()):   
        label.set_text(str(val))
        label.set_position((0,val))  
    
    plt.tick_params(bottom=True,top=True,left=True,right=True,labelbottom=True,labeltop=False,labelleft=True,labelright=False)
    ax.xaxis.set_visible(True)
    ax.yaxis.set_visible(True)
    ax.ticklabel_format(useOffset=False)
    ax.ticklabel_format(style='plain')
    ax.grid(False)
    ax.text(0.1, 0.9, u'\u25B2 \nN ',
        horizontalalignment='center',
        verticalalignment='center',
        fontsize=25, 
        color='k',
        family='Arial',
        transform=ax.transAxes)
    plt.title('KNN facies classification at %s m AHD (neighbours=%s)'%(slice_depth,n_neighbours), fontsize=20, weight='bold')

    df_1=df_slice[df_slice.Lithology_1 != ""]
    # X = df_1.as_matrix(columns=[EASTING_COL, NORTHING_COL])
    X = df_1[[EASTING_COL, NORTHING_COL]].values
    y = np.array(df_1[PRIMARY_LITHO_NUM_COL])
    knn = neighbors.KNeighborsClassifier(n_neighbours, weights = KNN_WEIGHTING).fit(X, y)
    grid_res=100
    
    # max/min bounds
    x_min=gw_subareas.total_bounds[0]
    y_min=gw_subareas.total_bounds[1]
    x_max=gw_subareas.total_bounds[2]
    y_max=gw_subareas.total_bounds[3]
    
    xx, yy = np.meshgrid(np.arange(x_min, x_max, grid_res),np.arange(y_min, y_max, grid_res))
    predicted = knn.predict(np.c_[xx.ravel(), yy.ravel()])
    predicted = predicted.reshape(xx.shape)
    plt.pcolormesh(xx, yy, predicted, cmap=cmap_settings['cmap'], norm=cmap_settings['norm'], alpha=0.3)
    ax.set_extent(near_field_extents, crs=data_proj)



In [ ]:
import cartopy.io.img_tiles as cimgt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
def bbox_as_list(bbox):
    return [bbox[i] for i in range(4)]

In [ ]:
extent = bbox_as_list(dem.bounds)
near_field_extents=bbox_as_list(dem.bounds)
#data_proj = dem.crs

In [ ]:
extent

In [ ]:
data_proj=ccrs.epsg(28355)

stamen_terrain = cimgt.Stamen('terrain-background')
terrain = stamen_terrain

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(15,15), subplot_kw={'projection': data_proj, 'extent': extent})
#fig,ax=plt.subplots(1,1,figsize=(15,15), subplot_kw={'projection': data_proj})
# fig.clear()
# ax.clear()
fig,ax=plt.subplots(1,1,figsize=(15,15), subplot_kw={'projection': data_proj, 'extent': extent})

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1, 1, 1, projection=data_proj)

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1, 1, 1, projection=data_proj)
#ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
#ax.get_extent()

In [ ]:
ax.set_extent(extent, data_proj)

In [ ]:
#ax.set_global()
#ax.stock_img()

ax.add_feature(cfeature.LAND, color='wheat')

ax.set_title('Cartopy Map Features - Demo', size=20, weight='bold', color='g')
ax.text(0.5, -0.06, 'Longitude', va='bottom', ha='center', size=15, color='r', rotation='horizontal', rotation_mode='anchor', transform=ax.transAxes)
ax.text(-0.02, 0.55, 'Latitude', va='bottom', ha='center', size=15, color='b', rotation='vertical', rotation_mode='anchor', transform=ax.transAxes)

plt.show()

In [ ]:
plt.show()

In [ ]:
ax.add_image(terrain, 11)

In [ ]:
#ax.add_geometries(geoms[0], ccrs.PlateCarree(),facecolor='none',edgecolor='k',zorder=1)
#ax.add_geometries(geoms[1], ccrs.PlateCarree(),facecolor='none',edgecolor='r',zorder=1)
for val,label in zip(ax.get_xticks(), ax.get_xticklabels()):
    label.set_text(str(val))
    label.set_position((val,0))  
for val,label in zip(ax.get_yticks(), ax.get_yticklabels()):   
    label.set_text(str(val))
    label.set_position((0,val))  
plt.tick_params(bottom=True,top=True,left=True,right=True,labelbottom=True,labeltop=False,labelleft=True,labelright=False)
ax.xaxis.set_visible(True)
ax.yaxis.set_visible(True)
ax.ticklabel_format(useOffset=False)
ax.ticklabel_format(style='plain')
ax.grid(False)
ax.text(0.1, 0.9, u'\u25B2 \nN ',
    horizontalalignment='center',
    verticalalignment='center',
    fontsize=25, 
    color='k',
    family='Arial',
    transform=ax.transAxes)
ax.set_extent(near_field_extents, crs=data_proj)

In [ ]:
ax.add_image(data_proj)

In [ ]:
slice_depth = 5.0
# Note that all of the above is independent of slice depth and background that would not need redoing
# but Matplotlib befuddles (or rather the interplay with ipywidgets)
df_slice=df.loc[(df[DEPTH_FROM_COL] <= slice_depth) & (df[DEPTH_TO_COL] >= slice_depth)]
ax.scatter(df_slice.Easting.values,df_slice.Northing.values)
plt.title('bore log locations at %s m depth'%(slice_depth), fontsize=20, weight='bold')
# I cannot fathom why this stuff actually plots anything 
# via ipywidgets or otherwise since it returns nothing.

In [ ]:

plot_lithologydata_slice_points_redo(df, 
    5.0, extent, data_proj, 
    near_field_extents, None, stamen_terrain)